In [1]:
#import libraries
from sklearn.externals.joblib import load
import pandas as pd
import math
from pandas import DataFrame
from rdkit.Chem.inchi import *
from rdkit import Chem
from sklearn.linear_model import LinearRegression
from rdkit.Chem import AllChem
import numpy as np
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import rdkit
# from tqdm import tqdm
from tqdm.notebook import tqdm
import xgboost as xgb
from pandas import Panel
tqdm.pandas()
import matplotlib.pyplot as plt
import os

C:\Users\Sergey\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Sergey\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
C:\Users\Sergey\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# function for descriptors using RDKIT ECFP
def morgan(mol, r=3, nBits=8128, errors_as_zeros=True):
    try:
        arr = np.zeros((1,))
        ConvertToNumpyArray(GetMorganFingerprintAsBitVect(mol, r, nBits), arr)
        return arr.astype(np.float32)
    except:
        return np.NaN if not errors_as_zeros else np.zeros((nBits,), dtype=np.float32)

In [3]:
def predict(input_file, output_file):
    #load models
    s=load('../models/std_scaler.bin')
    morg_bin= xgb.Booster()
    morg_reg = xgb.Booster()
    morg_bin.load_model('../models/morgan_bin.model')
    morg_reg.load_model('../models/morgan_reg.model') 
    #load data
    try:
        data = pd.read_csv(input_file, delimiter=";")
    except:
        data = pd.read_csv(input_file, delimiter=",")
    data['mol'] = data['InChI'].progress_apply(MolFromInchi)
    data['morgan'] = data['mol'].progress_apply(morgan)
    sample = xgb.DMatrix(np.vstack(data['morgan'].values))
    #get predictions
    pred_bin = morg_bin.predict(sample)
    pred_reg = np.array(morg_reg.predict(sample))
    pred_reg= s.inverse_transform(pred_reg).reshape(-1,1)
    #write output file
    data['Binary_retained'] = pred_bin
    data['Predicted_RT'] = pred_reg
    out = pd.DataFrame(list(zip(data['CID'], data['InChI'], data['Binary_retained'], data['Predicted_RT'])), columns=['CID', 'inchi', 'Binary_retained', 'Predicted_RT'])
    export_csv = out.to_csv (output_file, index = None, header=True, sep=';')
    
    

In [4]:
folders = os.listdir('../data/isomers/')

In [6]:
%%capture
folders = ['Eawag_XBridgeC18']
for folder in folders:
    os.makedirs('../data/predictions/isomers/'+folder, exist_ok=True)
    output_dir = '../data/predictions/isomers/'+folder+'/'
    path = '../data/isomers/'+folder+'/'
    for file in os.listdir(path):
        input_file = path+file
        print(input_file)
        output_file = output_dir+'pred_morg_'+file
        predict(input_file, output_file)